## 爬虫获取数据，并导出为dataframe格式

In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd

In [2]:
def extractor(xpath_text):
    '''根据xpath获取内容'''
    try:
        TCases = driver.find_element_by_xpath(xpath_text)
        return TCases.text
    except Exception as err:
        print(err)

In [3]:
ele_list = ['代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高价', '最低价', '今开',
                        '昨收', '量比', '换手率', '市盈率', '市净率']

In [4]:
df0 =  pd.DataFrame(columns=ele_list)

In [5]:
url = 'http://quote.eastmoney.com/center/gridlist.html#hs_a_board'
driver =webdriver.PhantomJS()
driver.get(url)
sleep(5)

F:\install\python\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [6]:
#沪深A
for page_num in range(1, 213):
    if page_num < 212:
        tmp_n = 11
    else:
        tmp_n = 10  #最后一页是17只股票
        
    for i in range(1, tmp_n):
        for ele_type in ['odd', 'even']:
            stock_dict = {}
            number_list = ['2', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
            for j, name in zip(number_list, ele_list):
                temp_xpath = "/html/body/div[@class='page-wrapper']/div[@id='page-body']/div[@id='body-main']/div[@id='table_wrapper']/div[@class='listview full']/table[@id='table_wrapper-table']/tbody/tr[@class='{}'][{}]/td[{}]".format(
                    ele_type, i, j)
                if extractor(temp_xpath):
                    stock_dict[name] = extractor(temp_xpath)
            if stock_dict:
                df0 = df0.append(stock_dict,ignore_index=True)
            if page_num == 212 and i == tmp_n-1:  #最后一页最后一个不再有even
                break
    #到下一页继续爬
    driver.find_element_by_xpath("/html/body/div[@class='page-wrapper']/div[@id='page-body']/div[@id='body-main']/div[@id='table_wrapper']/div[@class='listview full']/div[@class='dataTables_wrapper']/div[@id='main-table_paginate']/a[@class='next paginate_button']").click() 
    sleep(1)
    
driver.close()


Message: {"errorMessage":"Element is no longer attached to the DOM","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:58400","User-Agent":"selenium/3.141.0 (python windows)"},"httpVersion":"1.1","method":"GET","url":"/text","urlParsed":{"anchor":"","query":"","file":"text","directory":"/","path":"/text","relative":"/text","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/text","queryKey":{},"chunks":["text"]},"urlOriginal":"/session/dea023c0-36a6-11eb-bba3-a7cca12a13ea/element/:wdc:1607137622180/text"}}
Screenshot: available via screen

Message: {"errorMessage":"Element is no longer attached to the DOM","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:58400","User-Agent":"selenium/3.141.0 (python windows)"},"httpVersion":"1.1","method":"GET","url":"/t

In [7]:
df0.shape

(4237, 16)

## 修改数据类型并进行分析

In [9]:
import numpy as np

In [11]:
df0

,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高价,最低价,今开,昨收,量比,换手率,市盈率,市净率
0,601686,N友发,18.52,44.01%,5.66,4.81万,8895.16万,24.03%,18.52,15.43,15.43,12.86,-,3.38%,17.78,4.21
1,003021,N兆威,108.17,44.00%,33.05,6094,6576.76万,24.00%,108.17,90.14,90.14,75.12,-,2.29%,48.30,4.26
2,300858,科拓生物,72.95,20.00%,12.16,4.66万,3.12亿,18.34%,72.95,61.80,61.80,60.79,4.09,22.58%,62.24,6.63
3,300076,GQY视讯,7.08,15.31%,0.94,91.23万,6.45亿,21.66%,7.37,6.04,6.12,6.14,6.87,23.86%,182.14,2.92
4,300460,惠伦晶体,18.98,12.91%,2.17,42.56万,7.86亿,19.69%,19.86,16.55,16.55,16.81,1.92,18.06%,321.77,8.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4232,603377,东方时尚,19.87,-1.29%,-0.26,2.60万,5171.92万,4.47%,20.50,19.60,20.11,20.13,1.07,0.43%,108.25,6.41
4233,300030,阳普医疗,12.99,-1.29%,-0.17,6.27万,8149.11万,1.82%,13.16,12.92,13.16,13.16,1.07,2.47%,14.49,3.84
4234,600463,空港股份,6.87,-1.29%,-0.09,1.03万,706.54万,1.44%,6.95,6.85,6.94,6.96,0.60,0.34%,-57.57,1.67
4235,300022,吉峰科技,5.34,-1.29%,-0.07,10.75万,5764.33万,2.03%,5.44,5.33,5.39,5.41,0.97,3.11%,-283.67,25.81


In [12]:
df1=df0 

In [21]:
df1.columns

Index(['代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高价', '最低价', '今开',
       '昨收', '量比', '换手率', '市盈率', '市净率'],
      dtype='object')

In [13]:
df1.drop(['量比'],axis=1,inplace=True)

In [14]:
df2 = df1[~((df1['市盈率'].isin(['-']))|(df1['涨跌幅'].isin(['-']))|(df1['成交量'].isin(['-']))|(df1['最高价'].isin(['-']))|(df1['最低价'].isin(['-']))|(df1['换手率'].isin(['-'])))]

In [15]:
df2 

,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高价,最低价,今开,昨收,换手率,市盈率,市净率
0,601686,N友发,18.52,44.01%,5.66,4.81万,8895.16万,24.03%,18.52,15.43,15.43,12.86,3.38%,17.78,4.21
1,003021,N兆威,108.17,44.00%,33.05,6094,6576.76万,24.00%,108.17,90.14,90.14,75.12,2.29%,48.30,4.26
2,300858,科拓生物,72.95,20.00%,12.16,4.66万,3.12亿,18.34%,72.95,61.80,61.80,60.79,22.58%,62.24,6.63
3,300076,GQY视讯,7.08,15.31%,0.94,91.23万,6.45亿,21.66%,7.37,6.04,6.12,6.14,23.86%,182.14,2.92
4,300460,惠伦晶体,18.98,12.91%,2.17,42.56万,7.86亿,19.69%,19.86,16.55,16.55,16.81,18.06%,321.77,8.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4232,603377,东方时尚,19.87,-1.29%,-0.26,2.60万,5171.92万,4.47%,20.50,19.60,20.11,20.13,0.43%,108.25,6.41
4233,300030,阳普医疗,12.99,-1.29%,-0.17,6.27万,8149.11万,1.82%,13.16,12.92,13.16,13.16,2.47%,14.49,3.84
4234,600463,空港股份,6.87,-1.29%,-0.09,1.03万,706.54万,1.44%,6.95,6.85,6.94,6.96,0.34%,-57.57,1.67
4235,300022,吉峰科技,5.34,-1.29%,-0.07,10.75万,5764.33万,2.03%,5.44,5.33,5.39,5.41,3.11%,-283.67,25.81


将数据类型改为浮点数

In [16]:
df2[['最新价', '涨跌额', '最高价', '最低价', '今开','昨收', '市盈率', '市净率']] = df2[['最新价','涨跌额','最高价', '最低价', '今开','昨收','市盈率', '市净率']].apply(pd.to_numeric)

F:\install\python\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [17]:
df2['涨跌幅'] = df2['涨跌幅'].str.strip("%").astype(float)/100

F:\install\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df2['振幅'] = df2['振幅'].str.strip("%").astype(float)/100

F:\install\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df2['换手率'] = df2['换手率'].str.strip("%").astype(float)/100

F:\install\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df3  = df2[(df2['成交额'].notnull())&(df2['成交量'].notnull())]

In [21]:
df3

,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高价,最低价,今开,昨收,换手率,市盈率,市净率
0,601686,N友发,18.52,0.4401,5.66,4.81万,8895.16万,0.2403,18.52,15.43,15.43,12.86,0.0338,17.78,4.21
1,003021,N兆威,108.17,0.4400,33.05,6094,6576.76万,0.2400,108.17,90.14,90.14,75.12,0.0229,48.30,4.26
2,300858,科拓生物,72.95,0.2000,12.16,4.66万,3.12亿,0.1834,72.95,61.80,61.80,60.79,0.2258,62.24,6.63
3,300076,GQY视讯,7.08,0.1531,0.94,91.23万,6.45亿,0.2166,7.37,6.04,6.12,6.14,0.2386,182.14,2.92
4,300460,惠伦晶体,18.98,0.1291,2.17,42.56万,7.86亿,0.1969,19.86,16.55,16.55,16.81,0.1806,321.77,8.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4232,603377,东方时尚,19.87,-0.0129,-0.26,2.60万,5171.92万,0.0447,20.50,19.60,20.11,20.13,0.0043,108.25,6.41
4233,300030,阳普医疗,12.99,-0.0129,-0.17,6.27万,8149.11万,0.0182,13.16,12.92,13.16,13.16,0.0247,14.49,3.84
4234,600463,空港股份,6.87,-0.0129,-0.09,1.03万,706.54万,0.0144,6.95,6.85,6.94,6.96,0.0034,-57.57,1.67
4235,300022,吉峰科技,5.34,-0.0129,-0.07,10.75万,5764.33万,0.0203,5.44,5.33,5.39,5.41,0.0311,-283.67,25.81


In [22]:
df3.rename(columns={'成交量':'成交量（万）','成交额':'成交额（亿）'},inplace=True)

In [23]:
import re

In [24]:
def str_split(string):
    str_li = re.split(r'亿|万',string)[0]
    return str_li

In [25]:
df3['成交额（亿）'] = df3.apply(lambda df3:str_split(df3['成交额（亿）']),axis=1)

In [26]:
df3['成交额（亿）'] = df3['成交额（亿）'].astype('float')

In [27]:
df3['成交量（万）'] = df3.apply(lambda df3:str_split(df3['成交量（万）']),axis=1)

In [28]:
df3['成交量（万）'] = df3['成交量（万）'].astype('float')

In [31]:
df3['代码']

0       601686
1       003021
2       300858
3       300076
4       300460
         ...  
4232    603377
4233    300030
4234    600463
4235    300022
4236    300057
Name: 代码, Length: 4060, dtype: object

描述性统计分析

In [32]:
formater="{0:.02f}".format
df3.describe().applymap(formater)

,最新价,涨跌幅,涨跌额,成交量（万）,成交额（亿）,振幅,最高价,最低价,今开,昨收,换手率,市盈率,市净率
count,4060.00,4060.00,4058.00,4060.00,4060.00,4060.00,4060.00,4059.00,4060.00,4059.00,4060.00,4060.00,4060.00
mean,22.95,0.01,0.31,590.92,2570.20,0.03,23.25,22.44,22.66,22.66,0.02,64.79,4.38
std,47.04,0.02,1.47,1920.18,2858.64,0.02,47.50,45.75,46.00,46.03,0.03,736.75,14.64
min,0.15,-0.01,-5.10,1.00,1.00,0.00,0.15,0.14,0.15,0.14,0.00,-38244.01,-535.88
25%,6.46,-0.00,-0.02,3.11,3.30,0.02,6.55,6.34,6.39,6.41,0.01,16.73,1.79
50%,12.09,0.01,0.05,6.98,1578.76,0.02,12.32,11.83,11.92,11.94,0.01,37.26,3.11
75%,24.29,0.02,0.27,20.34,4497.48,0.04,24.50,23.74,23.98,23.95,0.02,70.17,5.12
max,1793.11,0.44,44.11,9978.00,9994.21,0.24,1800.10,1752.00,1752.00,1749.00,0.58,9945.76,434.98


相关性分析意义不大，可忽略

In [33]:
corr = df3.corr(method='pearson') 

In [34]:
corr

,最新价,涨跌幅,涨跌额,成交量（万）,成交额（亿）,振幅,最高价,最低价,今开,昨收,换手率,市盈率,市净率
最新价,1.000000,0.111950,0.701598,0.167352,-0.080658,0.108567,0.999744,0.999594,0.999571,0.999530,0.086561,-0.005408,0.109540
涨跌幅,0.111950,1.000000,0.489494,-0.079924,-0.085885,0.722653,0.112025,0.102257,0.101787,0.098665,0.349579,0.028028,0.033898
涨跌额,0.701598,0.489494,1.000000,0.025500,-0.081909,0.335307,0.700918,0.689493,0.687990,0.685196,0.149187,0.007494,0.069514
成交量（万）,0.167352,-0.079924,0.025500,1.000000,-0.046556,-0.130055,0.167354,0.169248,0.169981,0.170077,-0.089023,0.008879,0.059993
成交额（亿）,-0.080658,-0.085885,-0.081909,-0.046556,1.000000,-0.127704,-0.080986,-0.078971,-0.079057,-0.079203,-0.124577,0.008012,-0.021195
振幅,0.108567,0.722653,0.335307,-0.130055,-0.127704,1.000000,0.111906,0.099929,0.100815,0.100185,0.587067,0.020931,0.054641
最高价,0.999744,0.112025,0.700918,0.167354,-0.080986,0.111906,1.000000,0.999741,0.999747,0.999713,0.089466,-0.005382,0.109887
最低价,0.999594,0.102257,0.689493,0.169248,-0.078971,0.099929,0.999741,1.000000,0.999968,0.999924,0.082806,-0.005602,0.109346
今开,0.999571,0.101787,0.687990,0.169981,-0.079057,0.100815,0.999747,0.999968,1.000000,0.999950,0.083996,-0.005690,0.109738
昨收,0.999530,0.098665,0.685196,0.170077,-0.079203,0.100185,0.999713,0.999924,0.999950,1.000000,0.083511,-0.005791,0.109809
